<a href="https://colab.research.google.com/github/dnguyend/lagrange_rayleigh/blob/master/Eigen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

$\newcommand{\bx}{\boldsymbol{x}}$
$\newcommand{\bz}{\boldsymbol{z}}$
$\newcommand{\bH}{\boldsymbol{H}}$
$\newcommand{\bF}{\boldsymbol{F}}$
$\newcommand{\bI}{\boldsymbol{I}}$
$\newcommand{\bC}{\boldsymbol{C}}$
$\newcommand{\bJC}{\boldsymbol{J}_{\boldsymbol{C}}}$
$\newcommand{\PiH}{\boldsymbol{\Pi}_{\bH}}$
$\newcommand{\cR}{\mathcal{R}}$
$\newcommand{\bJR}{\bJ_{\mathcal{R}}}$
#Eigenvector example
This notebook illustrates Rayleigh/ Rayleigh Chebyshev quotient for the eigenvector example.

While the problem is the same, solving for
$$A \bx =\lambda \bx$$
In this system $\bF(\bx) = A\bx$, $\bH(x)= \bx$. The constraint to impose is $\bz^T \bx = 1$.
This constraint has been used in the past, and allows us to examine two different choices of the left inverse:
* $\bH^-(\bx) = (\bx^T\bx)^{-1}\bx$.
* $\bH^-(\bx) = \bz^T$.

It is easy to see they are both left inverses.
The following table summarizes the formula used in Rayleigh and Rayleigh Chebyshev interations



$\bH^-$|$\PiH$ |$\lambda$|$\bJR\eta$|Param. of Tangent | Param of $Im(\PiH)$|$\lambda_*$|
----------|----------|----------|----------|----------|----------|----------|
$(\bx^T\bx)^{-1}\bx$|$\bI -\bx\bx^T/(\bx^T\bx)$|$(\bx^T\bx)^{-1}\bx^T A\bx$|$(\bx^T\bx)^{-1}\bx^T (A+A^T)\eta-2(\bx^T\bx)^{-1}\lambda\bx^T\eta$|$Null(\bz^T)$|$Null(\bx^T)$|$(\bz^T\nu)/(\bz^T\eta)$|
$\bz^T$|$\bI -\bx\bz^T$|$\bz^TA\bx$|$\bz^TA\eta$|$Null(\bz^T)$|$Null(\bz^T)$|$(\bz^T\nu)/(\bz^T\eta)$|

For each choice of left inverse, we consider both the tangent and the ambient (schur) method. We also consider both the Rayleigh and Rayleigh Chebyshev iterations. The following codes define the $4$ functions of the iterations, with a flag to indicate if we choose to to Chebyshev iteration. We need a cutoff level of when to let Chebyshev step kick in. We will import our github then start running:

In [1]:
!git clone https://github.com/dnguyend/lagrange_rayleigh.git

fatal: destination path 'lagrange_rayleigh' already exists and is not an empty directory.


In [0]:
import numpy as np
from scipy.linalg import null_space
from numpy.linalg import solve, norm

def linear_rqi_xTx(x0, A, z, do_cheb=False,
                   max_iter=200, max_err=1e-8, cutoff=3e-1):
    """in this version use (x.Tx) for left inverse
    """
    x = x0.copy()
    k = A.shape[0]
    for i in range(max_iter):
        xTx = x.T @ x
        Ax = A @ x
        xTAx = x.T @ Ax
        lbd = xTAx / xTx
        zeta = solve(A - lbd * np.eye(k), x)
        err = norm(A @ x - lbd * x)
        # if err < max_err or (norm(zeta) * max_err > 10):
        if err < max_err:
            if err < max_err:
                return x, lbd, i, err, True
            else:
                return x, lbd, i, err, False

        nu = x + lbd * zeta
        lbd_s = (z.T @ nu) / (z.T @ zeta)
        eta = -nu + zeta * lbd_s

        if do_cheb and (norm(eta) < cutoff):
            J_R = -2*lbd/xTx*x@eta+x.T@(A.T+A)@eta/(xTx)
            T = solve(A - lbd * np.eye(k), eta * J_R)
            tau = T + eta - zeta / (z @ zeta) * (z @ (T+eta))
            x_n = (x + tau) / (z.T @ (x + tau))
        else:
            x_n = zeta / (z.T @ zeta)
        R = norm(x_n - x)
        x = x_n
        if R < max_err:
            break

    return x, lbd, i, err, err < max_err


def linear_rqi_zTx(x0, A, z, do_cheb=False,
                   max_iter=200, max_err=1e-8, cutoff=3e-1):
    """in this version use (z.Tx) for left inverse
    """
    x = x0.copy()
    k = A.shape[0]
    tau = None
    R = 1
    for i in range(max_iter):
        zTx = z.T @ x
        Ax = A @ x
        zTAx = z.T @ Ax
        lbd = zTAx / zTx
        zeta = solve(A - lbd * np.eye(k), x)
        err = norm(A @ x - lbd * x)
        # if err < max_err or (norm(zeta) * max_err > 10):
        if err < max_err:
            if err < max_err:
                return x, lbd, i, err, True
            else:
                return x, lbd, i, err, False
        nu = x + lbd * zeta
        lbd_s = (z.T @ nu) / (z.T @ zeta)
        eta = -nu + zeta * lbd_s

        if do_cheb and (norm(eta) < cutoff):
            J_R = -lbd*z.T@eta+z.T@A@eta
            T1 = solve(A - lbd * np.eye(k), eta * J_R)
            tau = T1 + eta - zeta / (z @ zeta) * (z @ (T1+eta))
            x_n = (x + tau) / (z.T @ (x+tau))
        else:
            x_n = zeta / (z.T @ zeta)
        R = norm(x-x_n)
        """
        print('i=%d tau=%s x=%s R=%f lbd=%f' % (
            i, str(tau), str(x), R, lbd))
        """
        x = x_n

        if R < max_err:
            break

        # if err < max_err:
        #    break
    return x, lbd, i, err, err < max_err


def linear_rqi_zTx_tangent(x0, A, z, do_cheb=False,
                           max_iter=200, max_err=1e-8, cutoff=3e-1):
    """in this version use (z.Tx) for left inverse
    z.T x = 1 is used for the constraint
    """
    # from scipy.linalg import null_space
    # Y = null_space(z)
    k = A.shape[0]
    Z = np.eye(k)
    # the Chebysev step:
    z = Z[:, 0]
    Y = Z[:, 1:]
    v = np.zeros((k-1))
    # y = Y u
    x_v = x0 / (z @ x0)
    success = False
    zTAY = z.T @ A @ Y
    tau = v
    err = 1
    for i in range(max_iter):
        x_v = x_v + Y @ tau
        p1 = np.eye(k) - x_v.reshape(-1, 1) @ z.reshape(1, -1)
        Y_p1 = Y.T @ p1
        lbd = z.T @ A @ x_v
        Rmd = (A @ x_v - lbd * x_v)
        err = norm(Rmd)
        if err <= max_err:
            success = True
            break
        pj_L = Y_p1 @ Rmd
        pj_Lx = Y_p1 @ (A - lbd * np.eye(k)) @ Y
        v = solve(pj_Lx, - pj_L)
        if do_cheb and (norm(v) < cutoff):
            J_R = zTAY @ v
            tau = v + solve(pj_Lx, Y_p1 @ Y @ v * J_R)
        else:
            tau = v
        
    if not success:
        x_v = x_v + Y @ v
        # p1 = np.eye(k) - x_v.reshape(-1, 1) @ z.reshape(1, -1)
        # Y_p1 = Y.T @ p1
        lbd = z.T @ A @ x_v
        Rmd = (A @ x_v - lbd * x_v)
        err = norm(Rmd)
        if err <= max_err:
            success = True
    # x, lbd, n_iter, err, converge
    return x_v, lbd, i, err, success


def linear_rqi_xTx_tangent(x0, A, z, do_cheb=False,
                           max_iter=200, max_err=1e-8, cutoff=3e-1):
    """(x.T x)^{-1} x.T is used for the left inverse
    z.T x = 1 is used for the constraint
    pH = I_k - (x x.T) / (x.T @ x)
    pH @ x_v = 0
    so a basis for pH is (null x_v.T).T
    """
    k = A.shape[0]
    Z = np.eye(k)
    # the Chebysev step:
    z = Z[:, 0]
    Y = Z[:, 1:]
    v = np.zeros((k-1))
    # y = Y u
    symAY = (A.T+A) @ Y
    x_v = x0 / (z @ x0)
    success = False
    tau = v
    err = 1
    for i in range(max_iter):
        x_v = x_v + Y @ tau
        xTx = x_v @ x_v
        pH = np.eye(k) - x_v.reshape(-1, 1) @ x_v.reshape(1, -1) / xTx
        YH = null_space(x_v.reshape(1, -1))
        Y_pH = YH.T @ pH
        lbd = (x_v.T @ A @ x_v) / xTx
        Rmd = (A @ x_v - lbd * x_v)
        err = norm(Rmd)
        if err <= max_err:
            success = True
            break
        pj_L = Y_pH @ Rmd
        pj_Lx = Y_pH @ (A - lbd * np.eye(k)) @ Y
        v = solve(pj_Lx, - pj_L)
        if do_cheb and (norm(v) < cutoff):
            J_R = -2*lbd/xTx*x_v@(Y@v)+x_v.T@symAY@v/(xTx)
            tau = v + solve(pj_Lx, Y_pH @ Y @ v * J_R)
        else:
            tau = v
        
    if not success:
        x_v = x_v + Y @ v
        # p1 = np.eye(k) - x_v.reshape(-1, 1) @ z.reshape(1, -1)
        # Y_p1 = Y.T @ p1
        xTx = x_v @ x_v
        lbd = x_v.T @ A @ x_v / xTx
        Rmd = (A @ x_v - lbd * x_v)
        err = norm(Rmd)
        if err <= max_err:
            success = True
    # x, lbd, n_iter, err, converge
    return x_v, lbd, i, err, success
        


The next function will run the test comparing all $8$ methods:

In [0]:
import pandas as pd
from IPython.core.display import HTML, display
from lagrange_rayleigh.core import utils
import time

def print_if_debug(E, debug):
  if debug:
    print(E)

def test_linear(k=50, n_test=500, debug=False):
    tbl2 = np.full((n_test, 32), np.nan)
    z = np.array([1] + (k-1)*[0])
    max_iter = 200
    max_err = 1e-10
    for i in range(n_test):
        A = utils.gen_random_real_eigen(k)
        x0 = np.random.randint(-5, 5, k) / 10.
        x0 = x0 / np.linalg.norm(x0)
        try:
            t_start = time.time()
            x, lbd, n_iter, err, converge = linear_rqi_xTx_tangent(
                x0, A, z, do_cheb=False, max_iter=max_iter,
                max_err=max_err, cutoff=3e-1)
            t_end = time.time()
            tbl2[i, 0] = n_iter
            tbl2[i, 1] = lbd
            tbl2[i, 2] = err
            tbl2[i, 3] = t_end - t_start
        except Exception as E:
            print_if_debug(E, debug)
        try:
            t_start = time.time()
            x, lbd, n_iter, err, converge = linear_rqi_xTx_tangent(
                x0, A, z, do_cheb=True, max_iter=max_iter,
                max_err=max_err, cutoff=10.e-1)
            t_end = time.time()
            tbl2[i, 4] = n_iter
            tbl2[i, 5] = lbd
            tbl2[i, 6] = err
            tbl2[i, 7] = t_end - t_start
        except Exception as E:
            print_if_debug(E, debug)
        try:
            t_start = time.time()
            x, lbd, n_iter, err, converge = linear_rqi_xTx(
                x0, A, z, do_cheb=False, max_iter=max_iter,
                max_err=max_err, cutoff=3e-1)
            t_end = time.time()
            tbl2[i, 8] = n_iter
            tbl2[i, 9] = lbd
            tbl2[i, 10] = err
            tbl2[i, 11] = t_end - t_start
        except Exception as E:
            print_if_debug(E, debug)
        try:
            t_start = time.time()
            x, lbd, n_iter, err, converge = linear_rqi_xTx(
                x0, A, z, do_cheb=True, max_iter=max_iter,
                max_err=max_err, cutoff=10.e-1)
            t_end = time.time()
            tbl2[i, 12] = n_iter
            tbl2[i, 13] = lbd
            tbl2[i, 14] = err
            tbl2[i, 15] = t_end - t_start
        except Exception as E:
            print_if_debug(E, debug)

        # zTx
        try:
            t_start = time.time()
            x, lbd, n_iter, err, converge = linear_rqi_zTx_tangent(
                x0, A, z, do_cheb=False, max_iter=max_iter,
                max_err=max_err, cutoff=3e-1)
            t_end = time.time()
            tbl2[i, 16] = n_iter
            tbl2[i, 17] = lbd
            tbl2[i, 18] = err
            tbl2[i, 19] = t_end - t_start
        except Exception as E:
            print_if_debug(E, debug)
        try:
            t_start = time.time()
            x, lbd, n_iter, err, converge = linear_rqi_zTx_tangent(
                x0, A, z, do_cheb=True, max_iter=max_iter,
                max_err=max_err, cutoff=5.e-1)
            t_end = time.time()
            tbl2[i, 20] = n_iter
            tbl2[i, 21] = lbd
            tbl2[i, 22] = err
            tbl2[i, 23] = t_end - t_start
        except Exception as E:
            print_if_debug(E, debug)
        try:
            t_start = time.time()
            x, lbd, n_iter, err, converge = linear_rqi_zTx(
                x0, A, z, do_cheb=False, max_iter=max_iter,
                max_err=max_err, cutoff=3e-1)
            t_end = time.time()
            tbl2[i, 24] = n_iter
            tbl2[i, 25] = lbd
            tbl2[i, 26] = err
            tbl2[i, 27] = t_end - t_start
        except Exception as E:
            print_if_debug(E, debug)
        try:
            t_start = time.time()
            x, lbd, n_iter, err, converge = linear_rqi_zTx(
                x0, A, z, do_cheb=True, max_iter=max_iter,
                max_err=max_err, cutoff=10.e-1)
            t_end = time.time()
            tbl2[i, 28] = n_iter
            tbl2[i, 29] = lbd
            tbl2[i, 30] = err
            tbl2[i, 31] = t_end - t_start
        except Exception as E:
            print_if_debug(E, debug)
        
    tbl3 = tbl2.copy()
    tbl3[np.where(~(tbl2[:, 2] < .05))[0], :4] = np.nan
    tbl3[np.where(~(tbl2[:, 6] < .05))[0], 4:8] = np.nan
    tbl3[np.where(~(tbl2[:, 10] < .05))[0], 8:12] = np.nan
    tbl3[np.where(~(tbl2[:, 14] < .05))[0], 12:16] = np.nan

    tbl3[np.where(~(tbl2[:, 18] < .05))[0], 16:20] = np.nan
    tbl3[np.where(~(tbl2[:, 22] < .05))[0], 20:24] = np.nan
    tbl3[np.where(~(tbl2[:, 26] < .05))[0], 24:28] = np.nan
    tbl3[np.where(~(tbl2[:, 30] < .05))[0], 28:32] = np.nan
    # print(np.nanmean(tbl3, axis=0))

    column0 = ['%s_iter %s_lbd %s_err %s_time' % (
        a, a, a, a) for a in [
            'xtx_tgt', 'xtx_chev_tgt', 'xtx_amb', 'xtx_chev_amb',
            'ztx_tgt', 'ztx_chev_tgt', 'ztx_amb', 'ztx_chev_amb',
        ]]
    columns = np.squeeze([a.split() for a in column0]).reshape(-1)
    pDF = pd.DataFrame(tbl3, columns=columns)
    for tag in ['iter', 'time', 'lbd', 'err' ]:
        display(HTML(pDF[[a for a in pDF.columns if a.endswith(tag)]].describe().to_html()))
    return pDF



A small test:

In [19]:
p1 = test_linear(k=5, n_test=100)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:158: RuntimeWarning: divide by zero encountered in true_divide
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:158: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:105: RuntimeWarning: divide by zero encountered in true_divide
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:105: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:112: RuntimeWarning: invalid value encountered in matmul
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:56: RuntimeWarning: divide by zero encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:57: RuntimeWarning: invalid value encountered in multiply
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:58: RuntimeWarning: invalid value encountered in multiply
/usr/local/lib/python3.6

,xtx_tgt_iter,xtx_chev_tgt_iter,xtx_amb_iter,xtx_chev_amb_iter,ztx_tgt_iter,ztx_chev_tgt_iter,ztx_amb_iter,ztx_chev_amb_iter
count,93.000000,93.000000,93.000000,93.000000,93.000000,93.000000,91.000000,91.000000
mean,6.000000,5.043011,5.989247,12.279570,7.118280,5.935484,7.021978,15.582418
std,1.817787,1.566733,1.832643,19.471476,2.151084,1.909965,2.200899,20.831102
min,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000
25%,5.000000,4.000000,5.000000,5.000000,5.000000,4.000000,5.000000,5.000000
50%,6.000000,5.000000,5.000000,7.000000,7.000000,6.000000,7.000000,7.000000
75%,7.000000,6.000000,7.000000,10.000000,8.000000,7.000000,8.000000,16.000000
max,12.000000,10.000000,12.000000,129.000000,16.000000,15.000000,16.000000,124.000000


,xtx_tgt_time,xtx_chev_tgt_time,xtx_amb_time,xtx_chev_amb_time,ztx_tgt_time,ztx_chev_tgt_time,ztx_amb_time,ztx_chev_amb_time
count,93.000000,93.000000,93.000000,93.000000,93.000000,93.000000,91.000000,91.000000
mean,0.001939,0.001752,0.000652,0.002170,0.000694,0.000735,0.000743,0.002822
std,0.000606,0.000492,0.000197,0.003242,0.000223,0.000250,0.000228,0.003790
min,0.001214,0.001046,0.000341,0.000531,0.000351,0.000377,0.000378,0.000573
25%,0.001589,0.001406,0.000514,0.000850,0.000536,0.000562,0.000573,0.000928
50%,0.001842,0.001630,0.000605,0.001336,0.000663,0.000670,0.000680,0.001382
75%,0.002157,0.002021,0.000765,0.001922,0.000780,0.000830,0.000867,0.002730
max,0.005442,0.003454,0.001415,0.020687,0.001676,0.001917,0.001436,0.021565


,xtx_tgt_lbd,xtx_chev_tgt_lbd,xtx_amb_lbd,xtx_chev_amb_lbd,ztx_tgt_lbd,ztx_chev_tgt_lbd,ztx_amb_lbd,ztx_chev_amb_lbd
count,9.300000e+01,9.300000e+01,93.000000,9.300000e+01,9.300000e+01,9.300000e+01,9.100000e+01,9.100000e+01
mean,-9.677419e-01,-9.677419e-01,-1.204301,-1.118280e+00,-6.344086e-01,-6.559140e-01,-5.164835e-01,-3.186813e-01
std,6.297777e+00,6.271834e+00,6.330669,6.317418e+00,6.693256e+00,6.650424e+00,6.692222e+00,6.644763e+00
min,-1.000000e+01,-1.000000e+01,-10.000000,-1.000000e+01,-1.000000e+01,-1.000000e+01,-1.000000e+01,-1.000000e+01
25%,-7.000000e+00,-6.000000e+00,-7.000000,-7.000000e+00,-7.000000e+00,-6.000000e+00,-6.500000e+00,-6.000000e+00
50%,-3.552714e-15,-1.299278e-15,-1.000000,-4.628584e-12,-3.552714e-15,-8.881784e-16,7.105427e-15,3.552714e-15
75%,4.000000e+00,4.000000e+00,4.000000,4.000000e+00,6.000000e+00,6.000000e+00,6.000000e+00,6.000000e+00
max,9.000000e+00,9.000000e+00,9.000000,9.000000e+00,9.000000e+00,9.000000e+00,9.000000e+00,9.000000e+00


,xtx_tgt_err,xtx_chev_tgt_err,xtx_amb_err,xtx_chev_amb_err,ztx_tgt_err,ztx_chev_tgt_err,ztx_amb_err,ztx_chev_amb_err
count,9.300000e+01,9.300000e+01,9.300000e+01,9.300000e+01,9.300000e+01,9.300000e+01,9.100000e+01,9.100000e+01
mean,4.508092e-12,4.463498e-12,6.630697e-12,3.358117e-08,6.459787e-12,2.414112e-12,2.595977e-11,1.546637e-08
std,1.442037e-11,1.540115e-11,2.221472e-11,9.672550e-08,1.669081e-11,1.003719e-11,7.199485e-11,3.559447e-08
min,6.841362e-16,6.206335e-16,1.275549e-15,9.918476e-16,6.684428e-16,4.577567e-16,2.482534e-16,2.141321e-15
25%,5.197931e-15,4.094300e-15,3.519600e-15,6.201258e-11,4.356830e-15,2.886580e-15,5.705450e-15,4.958400e-11
50%,2.927286e-14,9.969778e-15,3.892056e-14,8.667870e-10,1.699185e-14,8.157440e-15,4.380305e-14,4.116544e-10
75%,1.253768e-12,1.141024e-13,1.883032e-12,1.511032e-08,7.156159e-13,5.296404e-14,6.444967e-12,8.746563e-09
max,8.851630e-11,9.021067e-11,1.664152e-10,6.401542e-07,9.000303e-11,7.338873e-11,3.673435e-10,1.933458e-07


A bigger one:

In [24]:
test_linear(k=50, n_test=500)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:158: RuntimeWarning: divide by zero encountered in true_divide
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:158: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:105: RuntimeWarning: divide by zero encountered in true_divide
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:105: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:112: RuntimeWarning: invalid value encountered in matmul
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:113: RuntimeWarning: invalid value encountered in matmul
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:114: RuntimeWarning: invalid value encountered in matmul
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:115: RuntimeWarning: invalid value encountered in matmul
/usr/local/lib/python3.6/dist-pack

,xtx_tgt_iter,xtx_chev_tgt_iter,xtx_amb_iter,xtx_chev_amb_iter,ztx_tgt_iter,ztx_chev_tgt_iter,ztx_amb_iter,ztx_chev_amb_iter
count,437.000000,439.000000,493.000000,473.000000,447.000000,447.000000,447.000000,446.000000
mean,6.576659,6.091116,6.553753,40.405920,9.691275,9.357942,9.697987,62.024664
std,9.419709,9.421674,8.867570,48.028109,9.486998,13.093894,9.480713,52.206520
min,4.000000,3.000000,4.000000,4.000000,3.000000,3.000000,3.000000,4.000000
25%,5.000000,5.000000,5.000000,10.000000,7.000000,6.000000,7.000000,26.000000
50%,6.000000,5.000000,6.000000,23.000000,9.000000,8.000000,9.000000,44.000000
75%,7.000000,6.000000,7.000000,49.000000,11.000000,10.000000,11.000000,78.000000
max,199.000000,199.000000,199.000000,199.000000,199.000000,199.000000,199.000000,199.000000


,xtx_tgt_time,xtx_chev_tgt_time,xtx_amb_time,xtx_chev_amb_time,ztx_tgt_time,ztx_chev_tgt_time,ztx_amb_time,ztx_chev_amb_time
count,437.000000,439.000000,493.000000,473.000000,447.000000,447.000000,447.000000,446.000000
mean,0.003858,0.003835,0.001334,0.012909,0.002717,0.002914,0.001925,0.018062
std,0.004622,0.005011,0.001452,0.015909,0.002445,0.003593,0.001748,0.016753
min,0.002373,0.002141,0.000857,0.001008,0.001030,0.001178,0.000695,0.001314
25%,0.003031,0.003011,0.001045,0.002663,0.001976,0.002109,0.001386,0.006682
50%,0.003463,0.003304,0.001210,0.006733,0.002454,0.002513,0.001707,0.012029
75%,0.004013,0.004004,0.001402,0.015678,0.002950,0.003064,0.002046,0.022785
max,0.097884,0.105757,0.032683,0.069016,0.049525,0.057163,0.033643,0.075169


,xtx_tgt_lbd,xtx_chev_tgt_lbd,xtx_amb_lbd,xtx_chev_amb_lbd,ztx_tgt_lbd,ztx_chev_tgt_lbd,ztx_amb_lbd,ztx_chev_amb_lbd
count,437.000000,439.000000,493.000000,473.000000,447.000000,447.000000,447.000000,446.000000
mean,-0.720824,-0.676538,-0.555781,-0.729386,-0.340045,-0.333333,-0.342282,-0.244394
std,6.573847,6.585245,6.593868,6.569278,8.301266,8.288700,8.301849,8.317892
min,-10.000000,-10.000000,-10.000000,-10.000000,-10.000000,-10.000000,-10.000000,-10.000003
25%,-6.000000,-6.000000,-6.000000,-6.000000,-10.000000,-10.000000,-10.000000,-9.999999
50%,-1.000000,-1.000000,-1.000000,-1.000000,1.000000,1.000000,1.000000,1.500000
75%,5.000000,5.000000,5.000000,5.000000,8.000000,8.000000,8.000000,8.000000
max,9.000000,9.000000,9.000000,9.000029,9.000000,9.000000,9.000000,9.000699


,xtx_tgt_err,xtx_chev_tgt_err,xtx_amb_err,xtx_chev_amb_err,ztx_tgt_err,ztx_chev_tgt_err,ztx_amb_err,ztx_chev_amb_err
count,4.370000e+02,4.390000e+02,4.930000e+02,4.730000e+02,4.470000e+02,4.470000e+02,4.470000e+02,4.460000e+02
mean,1.353547e-05,1.772714e-06,1.255600e-11,5.636668e-05,1.266718e-08,4.499793e-06,2.499756e-10,1.343499e-04
std,2.829527e-04,3.714237e-05,1.153933e-10,9.958959e-04,2.676502e-07,9.501242e-05,5.118854e-09,1.761073e-03
min,1.615431e-14,1.943656e-14,2.655091e-14,2.196843e-14,2.612178e-14,2.065184e-14,4.087647e-14,5.083586e-14
25%,1.681422e-13,1.486758e-13,2.319110e-13,4.118545e-13,1.928283e-13,1.452178e-13,2.303010e-13,2.108737e-11
50%,5.373314e-13,4.495773e-13,6.525312e-13,6.548688e-12,5.973712e-13,4.064570e-13,7.092158e-13,4.457481e-11
75%,4.077905e-12,2.604274e-12,3.865100e-12,4.621428e-11,3.183317e-12,1.624395e-12,3.533288e-12,8.458744e-11
max,5.914997e-03,7.782190e-04,2.538338e-09,2.145681e-02,5.658769e-06,2.008792e-03,1.082318e-07,3.442688e-02


,xtx_tgt_iter,xtx_tgt_lbd,xtx_tgt_err,xtx_tgt_time,xtx_chev_tgt_iter,xtx_chev_tgt_lbd,xtx_chev_tgt_err,xtx_chev_tgt_time,xtx_amb_iter,xtx_amb_lbd,xtx_amb_err,xtx_amb_time,xtx_chev_amb_iter,xtx_chev_amb_lbd,xtx_chev_amb_err,xtx_chev_amb_time,ztx_tgt_iter,ztx_tgt_lbd,ztx_tgt_err,ztx_tgt_time,ztx_chev_tgt_iter,ztx_chev_tgt_lbd,ztx_chev_tgt_err,ztx_chev_tgt_time,ztx_amb_iter,ztx_amb_lbd,ztx_amb_err,ztx_amb_time,ztx_chev_amb_iter,ztx_chev_amb_lbd,ztx_chev_amb_err,ztx_chev_amb_time
0,6.0,-1.000000e+01,4.548867e-14,0.004962,6.0,-1.000000e+01,3.185093e-14,0.010172,6.0,-1.000000e+01,6.161632e-14,0.001250,176.0,-9.000000e+00,6.154104e-14,0.062588,7.0,9.0,3.179405e-12,0.001955,7.0,9.0,1.442932e-13,0.002346,7.0,9.0,3.238537e-12,0.001359,199.0,9.0,1.157978e-06,0.062205
1,4.0,1.150595e-14,2.569279e-13,0.002622,4.0,3.023844e-14,2.400994e-13,0.002673,4.0,-2.360906e-15,3.209417e-13,0.000861,4.0,-1.511543e-14,3.628062e-13,0.001171,15.0,5.0,9.941862e-11,0.003993,15.0,5.0,1.712818e-13,0.004885,15.0,5.0,9.941894e-11,0.002774,55.0,9.0,9.606191e-12,0.013580
2,5.0,-5.000000e+00,1.126942e-11,0.003042,4.0,-5.000000e+00,3.296855e-11,0.002729,5.0,-5.000000e+00,6.831930e-12,0.001039,33.0,-5.000000e+00,3.680691e-11,0.010302,11.0,-10.0,6.494214e-11,0.002938,11.0,-10.0,1.638417e-11,0.003261,12.0,-10.0,3.059491e-11,0.002223,136.0,-10.0,3.479161e-11,0.024933
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,3.000000e+00,3.539623e-12,0.001081,14.0,3.000000e+00,5.902552e-13,0.004137,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,7.0,-9.000000e+00,2.343485e-11,0.004017,7.0,-9.000000e+00,1.174350e-13,0.004138,7.0,-9.000000e+00,2.344973e-11,0.001367,52.0,-9.000000e+00,3.638966e-07,0.022653,10.0,-9.0,2.119926e-13,0.002758,9.0,-9.0,1.119234e-13,0.002794,10.0,-9.0,2.622751e-13,0.001864,26.0,-9.0,1.332505e-09,0.007902
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,5.0,-6.000000e+00,4.516944e-13,0.002955,5.0,-6.000000e+00,2.486915e-13,0.003304,5.0,-6.000000e+00,3.015722e-13,0.001043,22.0,-6.000000e+00,1.377644e-13,0.006661,7.0,9.0,4.307922e-13,0.001937,6.0,9.0,7.968988e-13,0.001945,7.0,9.0,6.248017e-13,0.001363,38.0,9.0,1.197804e-11,0.011043
496,5.0,3.000000e+00,5.822485e-14,0.002961,5.0,3.000000e+00,7.104956e-14,0.003299,5.0,3.000000e+00,7.433407e-14,0.001038,7.0,3.000000e+00,2.458411e-13,0.001878,5.0,-10.0,5.419083e-14,0.001419,4.0,-10.0,9.345975e-14,0.001443,5.0,-10.0,1.739611e-13,0.001042,45.0,-10.0,1.015604e-13,0.013241
497,9.0,-1.000000e+01,6.108808e-13,0.004838,9.0,-1.000000e+01,5.203725e-13,0.005136,9.0,-1.000000e+01,8.079179e-13,0.001724,36.0,-1.000000e+01,2.182861e-12,0.010514,11.0,9.0,8.993506e-12,0.002940,10.0,9.0,6.168314e-12,0.003052,11.0,9.0,2.072718e-12,0.002076,24.0,9.0,5.017994e-12,0.006029
498,5.0,4.000000e+00,5.596237e-14,0.002975,5.0,4.000000e+00,3.587113e-14,0.003199,5.0,4.000000e+00,4.740635e-14,0.001063,31.0,4.000000e+00,1.736437e-13,0.009467,6.0,6.0,8.546720e-12,0.001721,6.0,6.0,6.051173e-14,0.001963,6.0,6.0,8.571274e-12,0.001186,62.0,6.0,2.538762e-11,0.012764


In [9]:
print("A BIG ONE")
test_linear(k=100, n_test=500)

A BIG ONE


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:158: RuntimeWarning: divide by zero encountered in true_divide
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:158: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:105: RuntimeWarning: divide by zero encountered in true_divide
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:105: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:112: RuntimeWarning: invalid value encountered in matmul
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:113: RuntimeWarning: invalid value encountered in matmul
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:114: RuntimeWarning: invalid value encountered in matmul
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:115: RuntimeWarning: invalid value encountered in matmul
/usr/local/lib/python3.6/dist-pack

,xtx_tgt_iter,xtx_chev_tgt_iter,xtx_amb_iter,xtx_chev_amb_iter,ztx_tgt_iter,ztx_chev_tgt_iter,ztx_amb_iter,ztx_chev_amb_iter
count,447.000000,450.000000,500.000000,479.000000,451.000000,451.000000,451.000000,449.000000
mean,6.445190,5.642222,6.174000,28.227557,10.376940,9.108647,9.840355,51.677060
std,9.265928,1.883818,3.023575,25.244917,9.911376,3.032299,3.069023,36.782425
min,3.000000,3.000000,3.000000,3.000000,4.000000,3.000000,4.000000,5.000000
25%,5.000000,4.000000,5.000000,9.000000,8.000000,7.000000,8.000000,25.000000
50%,6.000000,5.000000,6.000000,20.000000,9.000000,9.000000,9.000000,41.000000
75%,7.000000,6.000000,7.000000,39.000000,12.000000,11.000000,12.000000,65.000000
max,199.000000,22.000000,53.000000,169.000000,199.000000,24.000000,24.000000,199.000000


,xtx_tgt_time,xtx_chev_tgt_time,xtx_amb_time,xtx_chev_amb_time,ztx_tgt_time,ztx_chev_tgt_time,ztx_amb_time,ztx_chev_amb_time
count,447.000000,450.000000,500.000000,479.000000,451.000000,451.000000,451.000000,449.000000
mean,0.015011,0.012743,0.004152,0.029156,0.013236,0.013265,0.006146,0.041542
std,0.016662,0.006503,0.002623,0.028826,0.009252,0.006006,0.002838,0.031387
min,0.005364,0.004956,0.001756,0.002120,0.003594,0.004318,0.002033,0.002987
25%,0.008874,0.007662,0.002460,0.008003,0.008283,0.008752,0.003972,0.017553
50%,0.013769,0.011045,0.003486,0.019380,0.011558,0.011545,0.005460,0.032729
75%,0.018252,0.015962,0.005132,0.040411,0.016359,0.016786,0.007850,0.057424
max,0.341553,0.066411,0.039412,0.172149,0.158666,0.036837,0.016710,0.198748


,xtx_tgt_lbd,xtx_chev_tgt_lbd,xtx_amb_lbd,xtx_chev_amb_lbd,ztx_tgt_lbd,ztx_chev_tgt_lbd,ztx_amb_lbd,ztx_chev_amb_lbd
count,4.470000e+02,4.500000e+02,500.000000,479.000000,451.000000,451.000000,451.000000,449.000000
mean,1.230425e-01,5.777778e-02,0.198000,0.179541,-0.110865,-0.119734,-0.110865,-0.006685
std,6.669695e+00,6.692831e+00,6.655177,6.677659,8.329927,8.333538,8.329927,8.359521
min,-1.000000e+01,-1.000000e+01,-10.000000,-10.000000,-10.000000,-10.000000,-10.000000,-10.000000
25%,-6.000000e+00,-6.000000e+00,-6.000000,-6.000000,-10.000000,-10.000000,-10.000000,-9.000000
50%,1.748083e-12,4.689376e-13,1.000000,1.000000,2.000000,2.000000,2.000000,3.000000
75%,6.000000e+00,6.000000e+00,6.000000,6.000000,9.000000,9.000000,9.000000,9.000000
max,9.000000e+00,9.000000e+00,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000


,xtx_tgt_err,xtx_chev_tgt_err,xtx_amb_err,xtx_chev_amb_err,ztx_tgt_err,ztx_chev_tgt_err,ztx_amb_err,ztx_chev_amb_err
count,4.470000e+02,4.500000e+02,5.000000e+02,4.790000e+02,4.510000e+02,4.510000e+02,4.510000e+02,4.490000e+02
mean,5.528849e-06,1.041230e-11,1.144925e-11,1.678644e-09,1.095278e-09,8.366722e-12,1.103146e-11,3.981307e-05
std,1.168927e-04,1.824438e-11,1.941867e-11,2.433584e-08,2.302790e-08,1.635599e-11,1.884130e-11,8.211139e-04
min,6.066100e-14,6.246547e-14,1.084531e-13,7.515089e-14,8.586787e-14,5.540112e-14,1.217728e-13,1.248776e-13
25%,7.885379e-13,7.479618e-13,9.370065e-13,8.021546e-13,6.324941e-13,5.490437e-13,7.951594e-13,2.095580e-11
50%,2.580738e-12,2.237191e-12,2.819757e-12,3.909604e-12,2.303531e-12,1.656021e-12,2.411612e-12,3.882982e-11
75%,1.231895e-11,9.909622e-12,1.140966e-11,2.896231e-11,1.214853e-11,6.635689e-12,1.110546e-11,6.668227e-11
max,2.471390e-03,9.534079e-11,9.812852e-11,4.970766e-07,4.890487e-07,9.595252e-11,9.818363e-11,1.739353e-02


In [20]:
p100 = test_linear(k=100, n_test=500)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:158: RuntimeWarning: divide by zero encountered in true_divide
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:158: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:33: RuntimeWarning: divide by zero encountered in true_divide
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:33: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in matmul
  del sys.path[0]
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in matmul
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:105: RuntimeWarning: divide by zero encountered in true_divide
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:105: RuntimeWarning: invalid value encountered in true_divide
/usr/loca

,xtx_tgt_iter,xtx_chev_tgt_iter,xtx_amb_iter,xtx_chev_amb_iter,ztx_tgt_iter,ztx_chev_tgt_iter,ztx_amb_iter,ztx_chev_amb_iter
count,446.000000,446.000000,497.000000,487.000000,455.000000,455.000000,455.000000,455.000000
mean,6.147982,5.957399,6.682093,28.425051,11.512088,10.406593,10.602198,51.630769
std,1.715909,5.429464,9.100001,31.396804,17.910428,15.730227,11.318477,37.502833
min,4.000000,3.000000,4.000000,3.000000,4.000000,3.000000,4.000000,4.000000
25%,5.000000,5.000000,5.000000,8.000000,8.000000,7.000000,8.000000,26.000000
50%,6.000000,5.000000,6.000000,18.000000,9.000000,9.000000,10.000000,42.000000
75%,7.000000,7.000000,7.000000,34.000000,12.000000,11.000000,12.000000,65.000000
max,15.000000,114.000000,199.000000,199.000000,199.000000,199.000000,199.000000,199.000000


,xtx_tgt_time,xtx_chev_tgt_time,xtx_amb_time,xtx_chev_amb_time,ztx_tgt_time,ztx_chev_tgt_time,ztx_amb_time,ztx_chev_amb_time
count,446.000000,446.000000,497.000000,487.000000,455.000000,455.000000,455.000000,455.000000
mean,0.012381,0.011324,0.003949,0.025881,0.012426,0.012538,0.005588,0.037527
std,0.005361,0.009450,0.005267,0.030418,0.014511,0.016194,0.004398,0.029243
min,0.004998,0.004837,0.001742,0.002083,0.003452,0.003656,0.001879,0.002385
25%,0.007562,0.007103,0.002327,0.005733,0.007530,0.007708,0.003508,0.015971
50%,0.011456,0.009289,0.003132,0.015157,0.009535,0.009690,0.004533,0.027890
75%,0.015788,0.013636,0.004475,0.033630,0.014415,0.014422,0.007177,0.050366
max,0.036172,0.180160,0.108569,0.206926,0.176250,0.219595,0.068400,0.191211


,xtx_tgt_lbd,xtx_chev_tgt_lbd,xtx_amb_lbd,xtx_chev_amb_lbd,ztx_tgt_lbd,ztx_chev_tgt_lbd,ztx_amb_lbd,ztx_chev_amb_lbd
count,4.460000e+02,4.460000e+02,4.970000e+02,4.870000e+02,455.000000,455.000000,455.000000,455.000000
mean,-4.573991e-01,-4.461883e-01,-3.138833e-01,-3.737168e-01,-0.142857,-0.182418,-0.142857,-0.151649
std,6.675579e+00,6.687269e+00,6.676426e+00,6.636038e+00,8.276568,8.278184,8.276568,8.247085
min,-1.000000e+01,-1.000000e+01,-1.000000e+01,-1.000000e+01,-10.000000,-10.000000,-10.000000,-10.000253
25%,-6.000000e+00,-6.000000e+00,-6.000000e+00,-6.000000e+00,-10.000000,-9.500000,-10.000000,-9.000000
50%,-6.770564e-14,-5.465505e-14,-2.882279e-14,-2.318225e-14,1.000000,1.000000,1.000000,1.000000
75%,5.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00,9.000000,9.000000,9.000000,9.000000
max,9.000000e+00,9.000000e+00,9.000000e+00,9.000000e+00,9.000000,9.000000,9.000000,9.000003


,xtx_tgt_err,xtx_chev_tgt_err,xtx_amb_err,xtx_chev_amb_err,ztx_tgt_err,ztx_chev_tgt_err,ztx_amb_err,ztx_chev_amb_err
count,4.460000e+02,4.460000e+02,4.970000e+02,4.870000e+02,4.550000e+02,4.550000e+02,4.550000e+02,4.550000e+02
mean,9.695773e-12,8.845319e-12,1.491385e-11,1.209238e-05,1.658735e-07,8.297169e-07,1.293416e-11,1.068639e-04
std,1.820075e-11,1.716897e-11,1.086980e-10,2.641000e-04,3.502615e-06,1.767095e-05,2.424788e-11,1.886805e-03
min,6.563903e-14,6.585757e-14,1.324587e-13,1.003377e-13,6.548186e-14,6.493760e-14,9.751686e-14,1.489308e-13
25%,7.356647e-13,5.553193e-13,7.985102e-13,8.240180e-13,6.927028e-13,6.241244e-13,8.880315e-13,2.097283e-11
50%,2.103237e-12,1.698541e-12,2.453448e-12,3.845130e-12,2.316266e-12,1.585641e-12,2.461917e-12,4.226995e-11
75%,8.434775e-12,6.655905e-12,9.083744e-12,3.057973e-11,1.044669e-11,5.898238e-12,1.141967e-11,6.752606e-11
max,9.837703e-11,9.988609e-11,2.399802e-09,5.828160e-03,7.471335e-05,3.769350e-04,2.240355e-10,3.937647e-02


In [23]:
p100.iloc[np.where(p100.xtx_amb_iter == 199)[0], :]

,xtx_tgt_iter,xtx_tgt_lbd,xtx_tgt_err,xtx_tgt_time,xtx_chev_tgt_iter,xtx_chev_tgt_lbd,xtx_chev_tgt_err,xtx_chev_tgt_time,xtx_amb_iter,xtx_amb_lbd,xtx_amb_err,xtx_amb_time,xtx_chev_amb_iter,xtx_chev_amb_lbd,xtx_chev_amb_err,xtx_chev_amb_time,ztx_tgt_iter,ztx_tgt_lbd,ztx_tgt_err,ztx_tgt_time,ztx_chev_tgt_iter,ztx_chev_tgt_lbd,ztx_chev_tgt_err,ztx_chev_tgt_time,ztx_amb_iter,ztx_amb_lbd,ztx_amb_err,ztx_amb_time,ztx_chev_amb_iter,ztx_chev_amb_lbd,ztx_chev_amb_err,ztx_chev_amb_time
353,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,199.0,9.0,2.399802e-09,0.108569,199.0,9.0,3.084731e-10,0.081469,199.0,-10.0,7.283604e-08,0.148212,199.0,-10.0,4.240022e-09,0.219149,199.0,-10.0,2.240355e-10,0.0684,199.0,-10.0,0.000054,0.07748
